Reference - this code is just modification of https://github.com/dennybritz/reinforcement-learning

In [1]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

In [2]:
import gym
import itertools
import matplotlib
import sys
import tensorflow as tf
import collections
from copy import deepcopy

matplotlib.style.use('ggplot')

# Load Data

In [3]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataLabeled\\poloniex\\5min'
fileName = 'USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
file = '{dataRoot}\\{fileName}'.format(dataRoot=dataRoot, fileName=fileName)

usdtBtc = pd.read_csv(file, sep='\t').query("date >= '2017-01-01'")\
    .sort_values('timestamp').reset_index().drop('index', axis=1)
usdtBtc = usdtBtc

trainRatio = 0.7
trainSize = int(usdtBtc.shape[0] * trainRatio)
usdtBtcTrain = usdtBtc.iloc[:trainSize]
usdtBtcTest = usdtBtc.iloc[trainSize:].reset_index().drop('index', axis=1)

usdtBtcManagerTrain = CurrencyDataManager(usdtBtcTrain['close'], usdtBtcTrain)
usdtBtcManagerTest = CurrencyDataManager(usdtBtcTest['close'], usdtBtcTest)
wallet = {'usdt': 1000}
contractPair = ContractPair('usdt', 'btc')

# Slicing Array

In [4]:
%%timeit
for i in range(1000):
    usdtBtc.iloc[10:11, :]

10 loops, best of 3: 131 ms per loop


In [15]:
%%timeit
for i in range(1000):
    usdtBtc.ix[10:11, :]

1 loop, best of 3: 276 ms per loop


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:258: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [14]:
%%timeit
arr = usdtBtc.values
for i in range(1000):
    arr[10:11, :]

1 loop, best of 3: 121 ms per loop


# Data Classes

In [5]:
from trading.money.transaction import SellTransaction
from trading.money.contract import ContractPair
from collections import namedtuple

In [6]:
%%timeit
cPair = ContractPair('usdt', 'btc')

The slowest run took 8.38 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 393 ns per loop


In [7]:
ContractPairNT = namedtuple('ContractPairNT', ['priceContract', 'tradeContract'])

In [8]:
%%timeit
tmp = ContractPairNT('usdt', 'btc')

The slowest run took 6.69 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 438 ns per loop


In [16]:
priceContract = 'usdt'
tradeContract = 'btc'
def contractPair(priceContract, tradeContract):
    return {'priceContract': priceContract, 'tradeContract': tradeContract}

In [17]:
%%timeit
tmp = contractPair(priceContract, tradeContract)

The slowest run took 5.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 206 ns per loop


In [18]:
%%timeit
tmp = {'priceContract': priceContract, 'tradeContract': tradeContract}

10000000 loops, best of 3: 96.6 ns per loop


In [20]:
class ContractPairTmp:
    @staticmethod
    def contractPair(priceContract, tradeContract):
        return {'priceContract': priceContract, 'tradeContract': tradeContract} 

In [21]:
%%timeit
tmp = ContractPairTmp.contractPair(priceContract, tradeContract)

The slowest run took 8.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 207 ns per loop


In [ ]:
tmp = Contract.new(name, value)

# Accesing object attribute vs dictionary key

In [1]:
class O:
    def __init__(self):
        self.contractPair = "usdt-btc"
        self.timestamp = 1
        self.amount = 10.
        self.price = 2.
        self.fee = .5

d = {'contractPair': "usdt-btc", 'timestamp': 1, 'amount': 10., 'price': 2., 'fee': .5}
o = O()

In [6]:
%%timeit
a = o.timestamp

The slowest run took 16.95 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 68.1 ns per loop


In [7]:
%%timeit
a = d['timestamp']

The slowest run took 19.77 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 58.4 ns per loop
